### Codio Activity 13.3: Determining the Logistic Functions Parameters

**Expected Time = 60 minutes** 

**Total Points = 50** 

This activity focuses on determining the appropriate parameters for the logistic model using optimization.  To begin, you will write a function to represent $\sigma(x)$.  Then, you will use the scikit-learn metric `log_loss` to evaluate the cross entropy based on the probabilities.  Finally, you will explore different parameters to select that which minimizes the cross entropy.

#### Index

- [Problem 1](#-Problem-1)
- [Problem 2](#-Problem-2)
- [Problem 3](#-Problem-3)
- [Problem 4](#-Problem-4)
- [Problem 5](#-Problem-5)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [2]:
penguins = sns.load_dataset('penguins').dropna()
penguins = penguins.loc[(penguins['species'] == 'Adelie') | (penguins['species'] == 'Gentoo')]
X = penguins.drop('species', axis = 1)[['flipper_length_mm']]
y = np.where(penguins.species == 'Adelie', 0, 1)

### The Problem

As discussed in the lectures, with gaussian assumptions of the data distributions, you are able to directly compute the parameters for the logistic model.  However, this is not going to work in higher dimensional settings.  As you see here, this assumption seems reasonable for the flipper length of our penguins data.

<center>
    <img src = 'images/flipperdist.png' />
</center>

But rather than using the mean and variance of these distributions learned through maximum likelihood, we will frame the problem of parameter estimation as one of minimization, and among a range of possible $\beta$ values, select that which minimized cross entropy.

[Back to top](#-Index)

### Problem 1

#### Sigma Function

**10 Points**

To begin, you are to define a sigmoid function with `sigma` below that takes in an array $X$, $\beta_0$, $\beta_1$, and returns 

$$\sigma(x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1x)}}$$

In [3]:
### GRADED

def sigma(x, beta_0, beta_1):
    """
    This function evaluates the sigmoid function with 
    given parameters beta_0 and beta_1
    
    Argments
    --------
    x: np.array
       domain for evaluation of sigma
    beta_0: float
       intercept term of linear function
    beta_1: float
        slope of linear term
    
    Returns
    -------
    np.array 
    """
    return None

### BEGIN SOLUTION
def sigma(x, beta_0, beta_1):
    zi = beta_0 + beta_1*x
    return 1/(1 + np.exp(-zi))
### END SOLUTION

# Answer check
# plt.plot(X.sort_values(by = 'flipper_length_mm'), 
#                sigma(X.sort_values(by = 'flipper_length_mm'), -165, .8), '--o')

In [4]:
### BEGIN HIDDEN TESTS
def sigma_(x, beta_0, beta_1):
    zi = beta_0 + beta_1*x
    return 1/(1 + np.exp(-zi))
stu = sigma(X, 1, 1)
ans = sigma(X, 1, 1)
#
#
#
np.testing.assert_array_equal(stu, ans)
### END HIDDEN TESTS

[Back to top](#-Index)

### Problem 2

#### Using a given $\beta_0$ and $\beta_1$ for predictions

**10 Points**

Now, your `sigma` function should return probabilities for making a classification.  Use your sigma function and the given $\beta_0$ and $\beta_1$ below to create predictions on X.  If the returned value is $\geq .5$ predict class 1 and otherwise predict 0. Assign the probabilites as an array to `probs` and the subsequent predictions based on these probabilities to `predictions`.  



In [5]:
beta_0 = -165
beta_1 = 0.7

In [6]:
### GRADED
probs = ''
predictions = ''


### BEGIN SOLUTION
probs = sigma(X, beta_0, beta_1)
predictions = np.where(probs >= 0.5, 1, 0)
### END SOLUTION

# Answer check
np.unique(predictions)

array([0])

In [7]:
### BEGIN HIDDEN TESTS
probs_ = sigma(X, beta_0, beta_1)
predictions_ = np.where(probs_ >= 0.5, 1, 0)
#
#
#
np.testing.assert_array_equal(predictions, predictions_)
### END HIDDEN TESTS

[Back to top](#-Index)

### Problem 3

#### Computing the Cross Entropy

**10 Points**

Recall that our goal is to find the $\beta_0$ and $\beta_1$ values that minimize the cross entropy.  The cross entropy was given to us as

$$-\sum_{i = 1} ^ N \big( (1 - y_i)\log(1 - \sigma(x_i)\big) + \big(y_i(\log \sigma(x_i) \big)$$

Scikitlearn has an implementation of this in the metrics module with the `log_loss` function (imported again below).  To use this, we want to pass an array of probabilities for both the positive and negative classes against the true $y$ values.  Below, this array is created for you using the earlier probabilities.  Compute the log loss of your predicted probabilities and assign the value to `loss1` below.  

In [8]:
from sklearn.metrics import log_loss

In [9]:
prob_array = np.concatenate((probs.values, (1 - probs).values), axis = 1)
pd.DataFrame(prob_array, columns = ['probability 0', 'probability 1']).head()

probability 0  probability 1
0   2.325527e-17            1.0
1   7.701087e-16            1.0
2   4.193796e-13            1.0
3   1.034177e-13            1.0
4   1.266417e-14            1.0

In [10]:
### GRADED

loss1 = ''

### BEGIN SOLUTION
loss1 = log_loss(y, prob_array)
### END SOLUTION

# Answer check
print(loss1)

17.157337553379403


In [11]:
### BEGIN HIDDEN TESTS
loss1_ = log_loss(y, prob_array)
#
#
#
assert loss1 == loss1_
### END HIDDEN TESTS

[Back to top](#-Index)

### Problem 4

#### Comparing Loss across Parameters

**10 Points**

Remember that your goal is to identify the parameters that **minimize** the log loss.  However, we know the rough values of this from our last assignment -- at least the values that come from the solved optimization problem with scikitlearn.  

Accordingly, below consider an array of parameters `beta_1s`.  Complete the code below to loop over these parameters, determine probabilities using them, create a probability array, and compute the log loss of these predictions.  

Keep track of the losses as a list in `losses`.  

Finally, determine the beta_1 that minimizes the log_loss and assign it as a float to `best_beta1`.

In [12]:
beta_1s = np.linspace(0, 1, 1000) #array of beta_1's to use
beta_0 = -142 #beta_0

In [13]:
### GRADED
losses = []
for beta_1 in beta_1s:
    probs = ''
    prob_array = ''
    losses.append(None)
    
best_beta1 = ''

### BEGIN SOLUTION
losses = []
for beta_1 in beta_1s:
    probs = sigma(X, beta_0, beta_1)
    prob_array = np.concatenate((probs.values, (1 - probs).values), axis = 1)
    losses.append(log_loss(y, prob_array))
    
best_beta1 = beta_1s[losses.index(min(losses))]
### END SOLUTION

# Answer check
print(best_beta1)

0.7087087087087087


In [14]:
### BEGIN HIDDEN TESTS
losses_ = []
for beta_1 in beta_1s:
    probs_ = sigma(X, beta_0, beta_1)
    prob_array_ = np.concatenate((probs_.values, (1 - probs_).values), axis = 1)
    losses_.append(log_loss(y, prob_array_))
    
best_beta1_ = beta_1s[losses_.index(min(losses_))]
#
#
#
assert best_beta1 == best_beta1_
### END HIDDEN TESTS

[Back to top](#-Index)

### Problem 5

#### Comparing the results to `LogisticRegression`

**10 Points**

Behind the scenes, this is what the `LogisticRegression` estimator is doing.  However, unlike our guess and test strategy above, a solver that implements gradient descent is used to target the optimal parameter values.  Specifically, the solvers `'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'` are available and `liblinear` is the default.  

As a check, implement a logistic regression estimator using scikitlearn with all default settings and examine the coefficient.  Is it close to what you selected in problem 4 above?  

Assign your fit estimator as `log_reg` the coefficient as a float to `coef` and the absolute difference between the coefficient from scikitlearn and that of `best_beta1` from problem 4 as difference.

In [15]:
### GRADED

logreg = ''
coef = ''
difference = ''

### BEGIN SOLUTION
logreg = LogisticRegression().fit(X, y)
coef = float(logreg.coef_)
difference = abs(coef - best_beta1)
### END SOLUTION

# Answer check
print(f'Beta_1 from sklearn: {coef}\nBeta_1 from our minimization: {best_beta1}\nDifference: {difference: .2f}')


Beta_1 from sklearn: 0.6948193765225441
Beta_1 from our minimization: 0.7087087087087087
Difference:  0.01


In [16]:
### BEGIN HIDDEN TESTS
logreg_ = LogisticRegression().fit(X, y)
coef_ = float(logreg_.coef_)
difference_ = abs(coef_ - best_beta1_)
#
#
#
assert coef == coef_
assert difference == difference_
### END HIDDEN TESTS